In [1]:
#Create human-readable, explainable fraud rules and assign a rule_score

In [15]:
import pandas as pd

In [16]:
import numpy as np
df=pd.read_csv("paysim_with_ml_scores.csv")
df.head()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,balance_delta_org,balance_delta_dest,hour,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER,ml_fraud_score,ml_anomaly_flag
0,278,330218.42,20866.00,351084.42,452419.57,122201.15,0,0,-330218.42,-330218.42,14,False,False,False,False,0.214322,0
1,15,11647.08,30370.00,18722.92,0.00,0.00,0,0,11647.08,0.00,15,False,False,True,False,0.007399,0
2,10,152264.21,106589.00,258853.21,201303.01,49038.80,0,0,-152264.21,-152264.21,10,False,False,False,False,0.124148,0
3,403,1551760.63,0.00,0.00,3198359.45,4750120.08,0,0,0.00,1551760.63,19,False,False,False,True,0.430251,0
4,206,78172.30,2921331.58,2999503.88,415821.90,337649.60,0,0,-78172.30,-78172.30,14,False,False,False,False,0.226824,0


In [17]:
df["rule_score"] = 0
df["rule_reasons"] = ""


In [18]:
balance_mismatch = abs(
    (df["oldbalanceOrg"] - df["newbalanceOrig"]) - df["amount"]
) > 1

df.loc[balance_mismatch, "rule_score"] += 40
df.loc[balance_mismatch, "rule_reasons"] += "Balance mismatch detected; "


In [19]:
high_risk_mask = (df["type_TRANSFER"] == True) | (df["type_CASH_OUT"] == True)

df.loc[high_risk_mask, "rule_score"] += 15
df.loc[high_risk_mask, "rule_reasons"] += (
    "Transaction type is historically associated with fraud (TRANSFER / CASH_OUT); "
)


In [20]:
# Using rolling average of amounts (no account grouping)
df["avg_amount"] = df["amount"].rolling(1000).mean()  # approximate historical trend
large_spike = df["amount"] > 3 * df["avg_amount"]

df.loc[large_spike, "rule_score"] += 25
df.loc[large_spike, "rule_reasons"] += "Transaction unusually large compared to typical amounts; "


In [21]:
df["rule_score"] = df["rule_score"].clip(0, 100)


In [22]:
df[["rule_score", "rule_reasons", "isFraud"]].sample(10)


,rule_score,rule_reasons,isFraud
6907,55,Balance mismatch detected; Transaction type is...,0
5113,0,,0
142202,55,Balance mismatch detected; Transaction type is...,0
105584,55,Balance mismatch detected; Transaction type is...,0
121228,55,Balance mismatch detected; Transaction type is...,0
135673,40,Balance mismatch detected;,0
18283,0,,0
90222,40,Balance mismatch detected;,0
147833,0,,0
118123,55,Balance mismatch detected; Transaction type is...,0


In [23]:
df['rule_reasons'].value_counts()

rule_reasons
Balance mismatch detected;                                                                                                                                            58818
Balance mismatch detected; Transaction type is historically associated with fraud (TRANSFER / CASH_OUT);                                                              52008
                                                                                                                                                                      25475
Balance mismatch detected; Transaction type is historically associated with fraud (TRANSFER / CASH_OUT); Transaction unusually large compared to typical amounts;      6702
Transaction type is historically associated with fraud (TRANSFER / CASH_OUT);                                                                                          6451
Balance mismatch detected; Transaction unusually large compared to typical amounts;                                            

In [24]:
df.to_csv(r"C:\Users\MAITHILI\Gen-AI_FraudGuard\paysim_with_rules.csv",
         index=False)